### All the different Datasets

In [1]:
#All the different Datasets

import requests
import csv

# token
token = 'zSeGtEWLAoSForkBhSGKwXVMFpErgCFb'

# Set the API endpoint URL
url = 'https://www.ncei.noaa.gov/cdo-web/api/v2/datasets'

params = {
    'format': 'json',  # Return the data in JSON format
}

# Set the authorization header 
headers = {'token': token}

# Send the GET request to the API and store the response
response = requests.get(url, params=params, headers=headers)

if response.status_code == 200:
    # Parse the response as JSON
    data = response.json()
            
    print('All the different datasets')
else:
    print(f'Request failed with status code {response.status_code}')
    print(response.text)
    

All the different datasets


In [2]:
print(response.json())


{'metadata': {'resultset': {'offset': 1, 'count': 11, 'limit': 25}}, 'results': [{'uid': 'gov.noaa.ncdc:C00861', 'mindate': '1763-01-01', 'maxdate': '2023-04-13', 'name': 'Daily Summaries', 'datacoverage': 1, 'id': 'GHCND'}, {'uid': 'gov.noaa.ncdc:C00946', 'mindate': '1763-01-01', 'maxdate': '2023-03-01', 'name': 'Global Summary of the Month', 'datacoverage': 1, 'id': 'GSOM'}, {'uid': 'gov.noaa.ncdc:C00947', 'mindate': '1763-01-01', 'maxdate': '2023-01-01', 'name': 'Global Summary of the Year', 'datacoverage': 1, 'id': 'GSOY'}, {'uid': 'gov.noaa.ncdc:C00345', 'mindate': '1991-06-05', 'maxdate': '2023-04-13', 'name': 'Weather Radar (Level II)', 'datacoverage': 0.95, 'id': 'NEXRAD2'}, {'uid': 'gov.noaa.ncdc:C00708', 'mindate': '1994-05-20', 'maxdate': '2023-04-14', 'name': 'Weather Radar (Level III)', 'datacoverage': 0.95, 'id': 'NEXRAD3'}, {'uid': 'gov.noaa.ncdc:C00821', 'mindate': '2010-01-01', 'maxdate': '2010-01-01', 'name': 'Normals Annual/Seasonal', 'datacoverage': 1, 'id': 'NORMAL

### All the different DataTypes

In [3]:
#All the different DataTypes

import requests
import csv

# token
token = 'zSeGtEWLAoSForkBhSGKwXVMFpErgCFb'

# API endpoint URL
url = 'https://www.ncei.noaa.gov/cdo-web/api/v2/locationcategories'
params = {
    'format': 'json',  # Return the data in JSON format
}
headers = {'token': token}

# Send GET request to the API and store the response
response = requests.get(url, params=params, headers=headers)


if response.status_code == 200:
    # Parse the response as JSON
    data = response.json()
            
    print('All the different datatypes')
else:
    print(f'Request failed with status code {response.status_code}')
    print(response.text)

All the different datatypes


In [4]:
print(response.json())


{'metadata': {'resultset': {'offset': 1, 'count': 12, 'limit': 25}}, 'results': [{'name': 'City', 'id': 'CITY'}, {'name': 'Climate Division', 'id': 'CLIM_DIV'}, {'name': 'Climate Region', 'id': 'CLIM_REG'}, {'name': 'Country', 'id': 'CNTRY'}, {'name': 'County', 'id': 'CNTY'}, {'name': 'Hydrologic Accounting Unit', 'id': 'HYD_ACC'}, {'name': 'Hydrologic Cataloging Unit', 'id': 'HYD_CAT'}, {'name': 'Hydrologic Region', 'id': 'HYD_REG'}, {'name': 'Hydrologic Subregion', 'id': 'HYD_SUB'}, {'name': 'State', 'id': 'ST'}, {'name': 'US Territory', 'id': 'US_TERR'}, {'name': 'Zip Code', 'id': 'ZIP'}]}


Will use the US territory as I need to research the US temperatures

In [6]:
from pymongo import MongoClient
import requests
import csv

# Set up a connection to the MongoDB server
client = MongoClient('mongodb://localhost:27000/')

# Choose the database and collection
db = client['mydatabase']
collection = db['mycollection']

# token
token = 'zSeGtEWLAoSForkBhSGKwXVMFpErgCFb'

# Set the API endpoint URL
url = 'https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND'

# Set the request parameters, such as location, date range, and data type
params = {
    'startdate': '2003-01-01',
    'enddate': '2022-01-31',
    'dataTypes': 'PRCP,TMAX,TMIN,US_TERR',  # Precipitation, maximum temperature, and minimum temperature
    'includeStationName': 'true',  
    'format': 'json',  
}

# Set the authorization header with your token
headers = {'token': token}

# Send the GET request to the API and store the response
response = requests.get(url, params=params, headers=headers)

if response.status_code == 200:
    # Parse the response as JSON
    data = response.json()
            
    print('All good')
else:
    print(f'Request failed with status code {response.status_code}')
    print(response.text)
    
# Insert the data into the database
collection.insert_many(response.json()['results'])


# Print the number of documents in the collection
print(f"Number of documents in collection: {collection.count_documents({})}")

All good
Number of documents in collection: 200


In [7]:
from pymongo import MongoClient

# Connect to the MongoDB server
client = MongoClient('mongodb://localhost:27000/')

# Select the database and collection
db = client['mydatabase']
collection = db['mycollection']

# Define the aggregation pipeline
pipeline = [
    {"$match": {"datatype": "TMAX"}},
    {"$group": {"_id": "$station", "avg_temp": {"$avg": "$value"}}},
    {"$sort": {"avg_temp": -1}}
]

# Execute the pipeline and print the results
result = collection.aggregate(pipeline)
for doc in result:
    print(doc)

{'_id': 'GHCND:AGE00147719', 'avg_temp': 201.0}
